<a href="https://colab.research.google.com/github/getuliomedeiros/SysSaida/blob/master/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn import datasets
from sklearn import svm
from sklearn.metrics import classification_report
from unicodedata import normalize
import unicodedata
import re
import keras 
from keras.datasets import imdb

Using TensorFlow backend.


In [ ]:
vocab_size = 10000
embedding_dim = 128
max_length = 500
trunc_type = 'post'
padding_type =  'post'
oov_tok = "<OOV>"
training_portion = .8

In [ ]:
sentences = []
labels = []
stopwords = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'a', 'com', 'nao', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'a', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'ha', 'nos', 'ja', 'esta', 'eu', 'tambam', 'sa3', 'pelo', 'pela', 'ata', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estao', 'vocaa', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'a s', 'minha', 'taam', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'sera', 'na3s', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocaas', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 
'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'esta', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estivaramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivassemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'ha', 'havemos', 'hao', 'houve', 'houvemos', 'houveram', 'houvera', 'houvaramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvassemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houvera', 'houveremos', 'houverao', 'houveria', 'houveraamos', 'houveriam', 'sou', 'somos', 'sao', 'era', 'aramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fa ramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fa ssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'sera', 'seremos', 'serao', 'seria', 'seraamos', 'seriam', 'tenho', 'tem', 'temos', 'tam', 'tinha', 'tanhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivaramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivassemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'tera', 'teremos', 'terao', 'teria', 'teraamos', 'teriam']

In [ ]:
def removerAcentosECaracteresEspeciais(palavra):

    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    palavraSemAcento = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', palavraSemAcento)
    palavraSemAcento = normalize('NFKD', palavraSemAcento).encode('ASCII','ignore').decode('ASCII').lower().rstrip()

    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

In [ ]:
with open('/content/drive/My Drive/IFPB/PROJETO/2020/classificados.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[6])
        sentence = row[7]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
            sentence = sentence.replace("  ", " ")
            sentence = removerAcentosECaracteresEspeciais(sentence)
        sentences.append(sentence)

    
print(len(labels))
print(len(sentences))
print(labels[0])
print(sentences[0])

4999
4999
1
ainda to palavras pra uniao pro dia hoje lindo ifc amo tiraamaodomeuif afederalluta


In [ ]:
labels.count('0')

587

In [ ]:
print(labels)
V_labels = []
for p in labels:
  valorLabel = int(p) + 1
  valorLabel = str(valorLabel)
  V_labels.append(valorLabel)
print(V_labels)

['1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '0', '-1', '-1', '-1', '0', '-1', '0', '0', '0', '0', '0', '-1', '-1', '0', '0', '0', '-1', '0', '-1', '-1', '-1', '-1', '0', '1', '0', '0', '-1', '0', '-1', '-1', '1', '-1', '0', '0', '0', '1', '-1', '1', '0', '1', '-1', '0', '-1', '-1', '-1', '0', '1', '1', '1', '0', '-1', '-1', '0', '0', '-1', '0', '-1', '-1', '-1', '-1', '0', '-1', '-1', '-1', '0', '-1', '-1', '-1', '1', '-1', '1', '-1', '0', '0', '0', '0', '-1', '0', '-1', '-1', '0', '-1', '-1', '-1', '0', '-1', '0', '-1', '1', '1', '1', '1', '1', '1', '1', '-1', '0', '1', '0', '-1', '-1', '0', '1', '0', '0', '0', '0', '0', '0', '-1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '-1', '0', '0', '0', '0', '0', '1', '0', '-1', '1', '1', '-1', '0', '0', '0', '1', '0', '-1', '0', '-1', '1', '0', '1', '0', '-1', '1', '0', '1', '1', '-1', '0', '1', '-1', '0', '0', '0', '0', '-1', '1', '1', '0', '0', '0', '0', '0', '1'

In [ ]:
labels = V_labels

In [ ]:
train_size = (int) (training_portion * len(labels))

train_sentences = sentences[0:train_size]
train_labels = labels[0:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_sentences))
print(len(train_labels))
print(len(validation_sentences))
print(len(validation_labels))

3999
3999
3999
1000
1000


In [ ]:
validation_labels.count('0')

54

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences,maxlen=max_length, padding=padding_type)

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

13
500
1
500
19
500


In [ ]:
print(train_sentences[2])
print(train_sequences[2])
print(word_index)
s = []
for i in train_sequences[2]:
  s.append(list(word_index.keys())[list(word_index.values()).index(i)])
print (s)
#print(word_index[item] for item in sequences[2])
print(train_padded[2])
print(train_padded.shape)

e incrivel ver quantidade estudante ainda preocupa educacao progresso brasil lutem messsssmoooo lotaram ruas parabens afederalluta tsunamidaeducacao todospelaeducacao
[49, 603, 66, 2097, 170, 68, 3540, 4, 2098, 16, 1465, 3541, 3542, 17, 252, 684, 2, 6]
{'<OOV>': 1, 'tsunamidaeducacao': 2, 'naruapelaeducacao': 3, 'educacao': 4, 'marolinha30m': 5, 'todospelaeducacao': 6, '15m': 7, '30mpelaeducacao': 8, 'meufilhonaovai': 9, 'dia': 10, 'pra': 11, '30m': 12, 'vai': 13, 'contra': 14, 'tsunami13a': 15, 'brasil': 16, 'ruas': 17, 'bolsonaro': 18, 'forabolsonaro': 19, 'amanha': 20, 'dia30meufilhonaovai': 21, 'manifestacao': 22, 'so': 23, 'balburdia': 24, 'todos': 25, 'tsunami30m': 26, 'estudantes': 27, 'hoje': 28, 'brasilpelaeducacao': 29, 'luta': 30, '13a': 31, 'governo': 32, 'dia30vaisermaior': 33, 'vamos': 34, 'rua': 35, 'pais': 36, 'o': 37, 'povo': 38, 'defesa': 39, 'jairbolsonaro': 40, 'cortes': 41, 'tiraamaodomeuif': 42, 'gente': 43, 'a': 44, '13deagosto': 45, 'ai': 46, 'lulalivre': 47, 't

In [ ]:
len(word_index)

9716

In [ ]:
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences,maxlen=max_length, padding=padding_type)

print(len(validation_sequences))
print(validation_padded.shape)

1000
(1000, 500)


In [ ]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

training_label_seq = array(train_labels)
validation_label_seq = array(validation_labels)

training_label_seq = to_categorical(training_label_seq, 3)
validation_label_seq = to_categorical(validation_label_seq, 3)

print(training_label_seq)
print(validation_label_seq)
print(type(training_label_seq))
print(type(validation_label_seq))

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
model = tf.keras.Sequential([
                             
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(3, activation='sigmoid')

])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 128)          1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 75        
Total params: 1,283,171
Trainable params: 1,283,171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# # loss='categorical_crossentropy'

In [ ]:
num_epochs = 30
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq))

Epoch 1/30
125/125 [==============================] - 2s 20ms/step - loss: 0.5970 - accuracy: 0.5359 - val_loss: 0.5174 - val_accuracy: 0.6210
Epoch 2/30
125/125 [==============================] - 2s 19ms/step - loss: 0.5714 - accuracy: 0.5359 - val_loss: 0.5226 - val_accuracy: 0.6210
Epoch 3/30
125/125 [==============================] - 2s 19ms/step - loss: 0.5713 - accuracy: 0.5359 - val_loss: 0.5155 - val_accuracy: 0.6210
Epoch 4/30
125/125 [==============================] - 2s 19ms/step - loss: 0.5697 - accuracy: 0.5359 - val_loss: 0.5184 - val_accuracy: 0.6210
Epoch 5/30
125/125 [==============================] - 2s 19ms/step - loss: 0.5684 - accuracy: 0.5359 - val_loss: 0.5082 - val_accuracy: 0.6210
Epoch 6/30
125/125 [==============================] - 2s 20ms/step - loss: 0.5665 - accuracy: 0.5359 - val_loss: 0.5099 - val_accuracy: 0.6210
Epoch 7/30
125/125 [==============================] - 2s 19ms/step - loss: 0.5629 - accuracy: 0.5359 - val_loss: 0.5210 - val_accuracy: 0.6210

In [ ]:
y_predictions = model.predict_classes(validation_padded)
# yy = tf.convert_to_tensor(y_predictions, dtype=tf.float32)
# print(y_predictions)

In [ ]:
xx_validation_labels = np.array(validation_labels)
x_validation_labels = tf.convert_to_tensor(xx_validation_labels, dtype=np.int32)
# print(x_validation_labels)

In [ ]:
confusion = tf.math.confusion_matrix(labels=x_validation_labels, predictions=y_predictions, num_classes=3)
print(confusion)

tf.Tensor(
[[  0  19  35]
 [  1 443 177]
 [  0  58 267]], shape=(3, 3), dtype=int32)


In [ ]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score, roc_curve

y_true = x_validation_labels
y_pred = y_predictions
print ("Precision", precision_score(y_true, y_pred, average='micro'))
print ("Recall", recall_score(y_true, y_pred, average='micro'))
print ("f1_score", f1_score(y_true, y_pred, average='micro'))
print ("confusion_matrix")
print (confusion_matrix(y_true, y_pred))
# fpr, tpr, tresholds = roc_curve(y_true, y_pred)

Precision 0.71
Recall 0.71
f1_score 0.7100000000000001
confusion_matrix
[[  0  19  35]
 [  1 443 177]
 [  0  58 267]]


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        54
           1       0.85      0.71      0.78       621
           2       0.56      0.82      0.66       325

    accuracy                           0.71      1000
   macro avg       0.47      0.51      0.48      1000
weighted avg       0.71      0.71      0.70      1000



In [ ]:
predictions = model.predict(train_padded)
predictions

array([[0.12412915, 0.0083113 , 0.9353391 ],
       [0.05980244, 0.84251904, 0.09505498],
       [0.07656488, 0.00209355, 0.978869  ],
       ...,
       [0.1628398 , 0.03113297, 0.8458562 ],
       [0.0384146 , 0.22187439, 0.6815538 ],
       [0.03069704, 0.6474263 , 0.3038245 ]], dtype=float32)

In [ ]:
predictions2 = model.predict_classes(train_padded)
predictions2

array([2, 1, 2, ..., 2, 2, 1])